In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\project Series\\Chest_Cancer_Classification_DL\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Data Science\\project Series\\Chest_Cancer_Classification_DL'

In [15]:
import dagshub
dagshub.init(repo_owner='qosain-bukhari', repo_name='Chest_CT_Scan_Classification_End_to_end_Project_MLOPS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2025-12-19 05:04:06,649 : INFO : httpx : HTTP Request: GET https://dagshub.com/api/v1/repos/qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS"

[2025-12-19 05:04:06,658 : INFO : dagshub : Initialized MLflow to track repo "qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS"]


Repository qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS initialized!

[2025-12-19 05:04:06,660 : INFO : dagshub : Repository qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS initialized!]
🏃 View run merciful-deer-757 at: https://dagshub.com/qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS.mlflow/#/experiments/1/runs/af3357d379d84330aa4a759346e53bcc
🧪 View experiment at: https://dagshub.com/qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS.mlflow/#/experiments/1


In [16]:
import tensorflow as tf
from pathlib import Path

In [17]:
trained_model_path = Path("artifacts/model_training/trained_model.h5")

In [18]:

# Load the trained model
trained_model_path = Path("artifacts/model_training/trained_model.h5")  # replace with your path
model = tf.keras.models.load_model(str(trained_model_path), compile=False)

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_model: Path
    training_data: Path
    all_params: dict
    mlflow_url:str
    params_image_size: list
    params_batch_size: int

In [20]:
from Cnnclassifier.constants import *
from Cnnclassifier.utils.common import read_yaml, create_directories

In [23]:

class ConfigManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_model=Path('artifacts/model_training/trained_model.h5'),  # Use Path
            training_data=Path('artifacts/data_ingestion/Chest-CT-Scan-data'),  # Fixed quotes
            mlflow_url="https://dagshub.com/qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS.mlflow",
            all_params=self.params,
            params_image_size=self.params['IMAGE_SIZE'],  # Access dict properly
            params_batch_size=self.params['BATCH_SIZE']  # Access dict properly
        )
        return eval_config


In [24]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from Cnnclassifier.utils.common import save_json

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluate_model(self):
        self.model = self.load_model(self.config.path_model)
        self._valid_generator()
        # Fixed: using self.model and capturing the evaluation score
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        score = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=score)
        print(f"score : {score}")

    def log_into_mlflow(self):
        """Logs parameters, metrics, and the model to DagsHub/MLflow."""
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            # Logs all parameters from your params.yaml
            mlflow.log_params(self.config.all_params)
            # Logs final evaluation metrics
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            # Model registry handling
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [25]:
try:
    config = ConfigManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate_model()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-12-19 05:04:46,459 : INFO : cnnclassifierlogger : yaml file: config\config.yaml loaded successfully]
[2025-12-19 05:04:46,466 : INFO : cnnclassifierlogger : yaml file: params.yaml loaded successfully]
[2025-12-19 05:04:46,470 : INFO : cnnclassifierlogger : Created directory at: artifacts]
[2025-12-19 05:04:46,606 : WARNING : absl : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 102 images belonging to 2 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 8.1810e-09   
[2025-12-19 05:04:54,183 : INFO : cnnclassifierlogger : json file saved at: scores.json]
score : {'loss': 8.181025634712569e-09, 'accuracy': 1.0}


2025/12/19 05:04:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/19 05:05:00 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


[2025-12-19 05:06:10,504 : WARNING : urllib3.connectionpool : Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine')': /qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS.mlflow/api/2.0/mlflow-artifacts/artifacts/e9278f747de64c409a9fb6f0365608db/models/m-a6b1c8a3153a448d9488c492bf61b19e/artifacts/data/model.keras]
[2025-12-19 05:08:21,531 : WARNING : urllib3.connectionpool : Retrying (Retry(total=5, connect=7, read=5, redirect=7, status=7)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /qosain-bukhari/Chest_CT_Scan_Classification_End_to_end_Project_MLOPS.mlflow/api/2.0/mlflow-artifacts/artifacts/e9278f747de64c409a9fb6f0365608db/models/m-a6b1c8a3153a448d9488c492bf61b19e/artifacts/data/model.keras]
[2025-12-19 05:12:15,853 : WARNING : u

KeyboardInterrupt: 